In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import gpflow
import gpflow.multikernels as mk

In [2]:
X = np.random.rand(100)[:, None] * 10 - 5
G = np.hstack((0.5 * np.sin(3 * X) + X, 3.0 * np.cos(X) - X))
Ptrue = np.array([[0.5, -0.3, 1.5], [-0.4, 0.43, 0.0]])
Y = np.matmul(G, Ptrue)
Y += np.random.randn(*Y.shape) * [0.2, 0.2, 0.2]

In [3]:
def plot_model(m):
    pX = np.linspace(-6, 6, 100)[:, None]
    pY, pYv = m.predict_y(pX)
    if pY.ndim == 3:
        pY = pY[:, 0, :]
    plt.plot(m.X.value, m.Y.value, 'x')
    plt.gca().set_color_cycle(None)
    plt.plot(pX, pY)
    for i in range(pY.shape[1]):
        top = pY[:, i] + 2.0 * pYv[:, i] ** 0.5
        bot = pY[:, i] - 2.0 * pYv[:, i] ** 0.5
        plt.fill_between(pX[:, 0], top, bot, alpha=0.3)

# MixedMultiOutput kernel -- Independent inducing points

In [4]:
M = 20
K = 3
L = 2
R = 1
q_mu = np.zeros((M, L, R))
q_sqrt = np.tile(np.eye(M)[None, None, :, :], [R, L, 1, 1])
with gpflow.defer_build():
    kern = mk.MixedMulti([gpflow.kernels.RBF(1) + gpflow.kernels.Linear(1) for _ in range(L)], P=Ptrue.T)
    feature = mk.MixedMultiIndependentFeature(Z=X[:M, :].copy())
    m = gpflow.models.SVGP(X, Y, kern, gpflow.likelihoods.Gaussian(), feature)
    m.q_mu = gpflow.params.Parameter(q_mu, dtype=gpflow.settings.float_type)
    m.q_sqrt = gpflow.params.Parameter(q_sqrt, dtype=gpflow.settings.float_type)
m.compile()

In [5]:
opt = gpflow.train.ScipyOptimizer()
opt.minimize(m, disp=True, maxiter=30000)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -16.236387
  Number of iterations: 10080
  Number of functions evaluations: 10732


In [6]:
m.compute_log_likelihood()

16.23638719832087